In [2]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("No GPU available, falling back to CPU.")


No GPU available, falling back to CPU.


In [ ]:
from mesh_video_generator import MeshVideoGenerator

hw = 32
num_views = 2
data_dirs = ["meshes", "SHREC19_MH_dataset", "SHREC20b_hires/models", "MPI-FAUST/training/scans"]

generator = MeshVideoGenerator(
    output_dir=data_dirs[0],
    hw=hw,
    num_views=num_views,
    use_normal_map=True,
    device=device
)
# generator.process_folder("meshes", display_frames=False)
generator.process_single_mesh("meshes", index=5)

In [ ]:
import torch
from diff3f import get_features_per_vertex

from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer

from sam2_setup import init_sam2
from functional_map import compute_surface_map

/opt/homebrew/anaconda3/envs/diff3f/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/anaconda3/envs/diff3f/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /opt/homebrew/anaconda3/envs/diff3f/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/opt/homebrew/anaconda3/envs/diff3f/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/homebrew/anaconda3/envs/diff3f/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/homebrew/anaconda3/envs/diff3f/lib/python3.10/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/homebrew/anaconda3/envs/diff3f/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be

ModuleNotFoundError: No module named 'sam2.build_sam'; 'sam2' is not a package

In [ ]:
device = torch.device('cuda:0')
torch.cuda.set_device(device)
num_views = 2
H = 32
W = 32
num_images_per_prompt = 1
tolerance = 0.004
random_seed = 42
use_normal_map = True

In [ ]:
def compute_features(device, sam_model, mesh, prompt, num_views, H, W):
    mesh = convert_mesh_container_to_torch_mesh(mesh, device=device, is_tosca=False)
    mesh_vertices = mesh.verts_list()[0]
    features = get_features_per_vertex(
        device=device,
        sam_model=sam_model,
        mesh=mesh,
        prompt=prompt,
        mesh_vertices=mesh_vertices,
        num_views=num_views,
        H=H,
        W=W,
        tolerance=tolerance,
        use_normal_map=use_normal_map
    )
    return features.cpu()

In [ ]:
sam_model = init_sam2(device)

In [ ]:
source_file_path = "meshes/cow.obj"
target_file_path = "meshes/camel.obj"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)

In [ ]:
f_source = compute_features(device, sam_model, target_mesh, "cow")
f_target = compute_features(device, sam_model, target_mesh, "camel")

In [ ]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  